In [ ]:
from google.colab import drive
drive.mount('/gdrive/')

In [ ]:
train= pd.read_csv('/gdrive/My Drive/nmt/data/train.csv', error_bad_lines=False)
val= pd.read_csv('/gdrive/My Drive/nmt/data/val.csv', error_bad_lines=False)
labels= pd.read_csv('/gdrive/My Drive/nmt/data/labels.csv', error_bad_lines=False)

In [ ]:
from fast_bert.data_cls import BertDataBunch

databunch = BertDataBunch(DATA_PATH, LABEL_PATH,
                          tokenizer='bert-base-uncased',
                          train_file='train.csv',
                          val_file='val.csv',
                          label_file='labels.csv',
                          text_col='text',
                          label_col='label',
                          batch_size_per_gpu=16,
                          max_seq_length=512,
                          multi_gpu=True,
                          multi_label=False,
                          model_type='bert')

In [ ]:
# Language model Databunch
from fast_bert.data_lm import BertLMDataBunch
# Language model learner
from fast_bert.learner_lm import BertLMLearner

from pathlib import Path
from box import Box

In [ ]:
# # Box is a nice wrapper to create an object from a json dict
# args = Box({
#     "seed": 42,
#     "task_name": 'imdb_reviews_lm',
#     "model_name": 'roberta-base',
#     "model_type": 'roberta',
#     "train_batch_size": 16,
#     "learning_rate": 4e-5,
#     "num_train_epochs": 20,
#     "fp16": True,
#     "fp16_opt_level": "O2",
#     "warmup_steps": 1000,
#     "logging_steps": 0,
#     "max_seq_length": 512,
#     "multi_gpu": True if torch.cuda.device_count() > 1 else False
# })

DATA_PATH = Path('../lm_data/')
LOG_PATH = Path('../logs')
MODEL_PATH = Path('../lm_model_{}/'.format(args.model_type))

DATA_PATH.mkdir(exist_ok=True)
MODEL_PATH.mkdir(exist_ok=True)
LOG_PATH.mkdir(exist_ok=True)

In [ ]:
from fast_bert.learner_cls import BertLearner
from fast_bert.metrics import accuracy
import logging

logger = logging.getLogger()
device_cuda = torch.device("cuda")
metrics = [{'name': 'accuracy', 'function': accuracy}]

learner = BertLearner.from_pretrained_model(
						databunch,
						pretrained_path='bert-base-uncased',
						metrics=metrics,
						device=device_cuda,
						logger=logger,
						output_dir=OUTPUT_DIR,
						finetuned_wgts_path=None,
						warmup_steps=500,
						multi_gpu=True,
						is_fp16=True,
						multi_label=False,
						logging_steps=50)

In [ ]:
learner.lr_find(start_lr=1e-5,optimizer_type='lamb')

In [ ]:
learner.fit(epochs=6,
			lr=6e-5,
			validate=True, 	# Evaluate the model after each epoch
			schedule_type="warmup_cosine",
			optimizer_type="lamb")

In [ ]:
learner.save_model()

In [ ]:
from fast_bert.prediction import BertClassificationPredictor

MODEL_PATH = OUTPUT_DIR/'model_out'

predictor = BertClassificationPredictor(
				model_path=MODEL_PATH,
				label_path=LABEL_PATH, # location for labels.csv file
				multi_label=False,
				model_type='xlnet',
				do_lower_case=False)

multiple_predictions = predictor.predict_batch("text.csv")